# Retrieve detailed information about each blockface

In section 1.1, I collected a list of blockfaces that have paid parking. Here, I collect detailed information for each of these blockfaces, including street name, cross streets, and latitude and longitude. Requests are made using the socrata API with the city of seattle data (https://data.seattle.gov/Transportation/2019-Paid-Parking-Occupancy-Year-to-date-/qktt-2bsy). Data is output to a csv (blockface_detail.csv).

In [73]:
# Import modules
import pandas as pd
from datetime import timedelta, date
from sodapy import Socrata
import requests

In [74]:
# import app tokens, secret key for API calls
import os, sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
import app_tokens

I define a method to retrieve a single record for a given blockface ID (**sourceelementkey**)

In [75]:
def getOneLocation(client, data_key, elementkey):

    results = client.get(data_key,
                     select='blockfacename,sideofstreet,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,sourceelementkey,location',
                     sourceelementkey=elementkey,
                     limit=1)
    return results

In [76]:
#data is delayed 48 hrs
# socrata data keys for parking data
#2019 ytd
data_ytd = 'qktt-2bsy'
#last 30 days
data_mtd = 'rke9-rsvs'
# last 48 hours
data_48hrs = 'hiyf-7edq'


In [77]:
# Read in list of sourceelementkeys
df = pd.read_csv('block_locations.csv')

In [78]:
df['sourceelementkey'].values

array([  1001,   1002,   1006, ..., 135261, 136041, 136322])

In [79]:
all_results = []

#initialize client
client = Socrata('data.seattle.gov',
                 app_tokens.getAppTokens()['seattle_gov'],
                timeout=60)
start = 0
end = len(df)
# loop through possible values
for index, val in df['sourceelementkey'].iteritems():    
    print('i: %5d\tpercent done : %.3f' % (index, 100*(index - start) /(end-start)), end='\r')
    results = getOneLocation(client=client, data_key=data_ytd, elementkey=val)
    #print(results)
    if len(results) > 0:
        all_results.append(results[0])

# close connection
client.close()

In [80]:
#convert data to pandas dataframe
df_all = pd.DataFrame.from_records(all_results)

In [81]:
# inspect elements
df_all.head()

,blockfacename,location,paidparkingarea,paidparkingsubarea,parkingcategory,parkingspacecount,parkingtimelimitcategory,sideofstreet,sourceelementkey
0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3346935...",Pioneer Square,Core,Paid Parking,4,120,SW,1001
1,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3345126...",Pioneer Square,Core,Paid Parking,8,120,NE,1002
2,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,"{'type': 'Point', 'coordinates': [-122.3351432...",Commercial Core,Waterfront,Paid Parking,7,120,NE,1006
3,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3366575...",Commercial Core,Waterfront,Paid Parking,5,30,SW,1009
4,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3364474...",Commercial Core,Waterfront,Paid Parking,5,120,NE,1010


In [82]:
# unpack location object into latitude and longitude
def unpackCoordinates(location_obj):
    lat = location_obj['coordinates'][0]
    long = location_obj['coordinates'][1]
    # all seattle lat/long should be approx 47, -122. swap lat and long if wrong way
    if lat < long:
        return pd.Series([long, lat])
    else:
        return pd.Series([lat, long])

In [83]:
#convert records from strings to numeric types
df_all[['sourceelementkey', 'parkingspacecount', 'parkingtimelimitcategory']] = df_all[
    ['sourceelementkey', 'parkingspacecount', 'parkingtimelimitcategory']].apply(pd.to_numeric)
# sort by sourceelementkey
df_all = df_all.sort_values(by='sourceelementkey')
#unpack location object into latitude and longitude columns
df_all[['latitude', 'longitude']] = df_all.location.apply(unpackCoordinates)

In [84]:
df_all.head()

,blockfacename,location,paidparkingarea,paidparkingsubarea,parkingcategory,parkingspacecount,parkingtimelimitcategory,sideofstreet,sourceelementkey,latitude,longitude
0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3346935...",Pioneer Square,Core,Paid Parking,4,120.0,SW,1001,47.602873,-122.334694
1,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3345126...",Pioneer Square,Core,Paid Parking,8,120.0,NE,1002,47.602949,-122.334513
2,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,"{'type': 'Point', 'coordinates': [-122.3351432...",Commercial Core,Waterfront,Paid Parking,7,120.0,NE,1006,47.603674,-122.335143
3,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3366575...",Commercial Core,Waterfront,Paid Parking,5,30.0,SW,1009,47.605018,-122.336658
4,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3364474...",Commercial Core,Waterfront,Paid Parking,5,120.0,NE,1010,47.605101,-122.336447


In [85]:
df_all.to_csv('blockface_detail.csv')

In [89]:
#Calculate number of available parking spaces
parking_zones = df_all.groupby(['paidparkingarea']).count()['blockfacename'].rename('numberparkingzones')
parking_spaces = df_all.groupby(['paidparkingarea']).sum()['parkingspacecount'].rename('numberparkingspaces')
parkingAreaInfo = pd.concat([parking_zones, parking_spaces], axis=1)
parkingAreaInfo['numberparkingspaces'].sum()

11907